In [35]:
from flask import Flask, redirect, url_for, render_template, request
from keras.models import load_model
import pickle
from nltk.tokenize import RegexpTokenizer
import numpy as np
import heapq

model = pickle.load(open('NextWord.pkl', 'rb'))

path = 'data.txt'
text = open(path, encoding="utf8").read().lower()

# Tokenization

tokenizer = RegexpTokenizer(r'\w+')
words = tokenizer.tokenize(text)

# Getting unique words

unique_words = np.unique(words)
unique_word_index = dict((c, i) for i, c in enumerate(unique_words))

LENGTH_WORD = 5

# Testing Next Word
def prepare_input(text):
    x = np.zeros((1, LENGTH_WORD, len(unique_words)))
    for t, word in enumerate(text.split()):
#        print(word)
        try:
            x[0, t, unique_word_index[word]] = 1
        except:
            pass
    return x

def sample(preds, top_n):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds)
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    return heapq.nlargest(top_n, range(len(preds)), preds.take)

def predict_completions(text, n):
    x = prepare_input(text)
    preds = model.predict(x, verbose=0)[0]
    next_indices = sample(preds, n)
    return [unique_words[idx] for idx in next_indices]


app = Flask(__name__)

@app.route("/")
def home():
    return render_template("index.html")

@app.route("/main", methods=["POST", "GET"])
def main():
    if request.method == "POST":
        input_text = request.form["text"]
        x = prepare_input(input_text)
        preds = model.predict(x, verbose=0)[0]
        next_indices = sample(preds, 10)
        suggestions = predict_completions(input_text, 10)
        string = suggestions[0] + ', ' + suggestions[1] + ', ' + suggestions[2] + ', ' + suggestions[3] + ', ' + suggestions[4] + ', ' + suggestions[5] + ', ' + suggestions[6] + ', ' + suggestions[7] + ', ' + suggestions[8] + ', ' + suggestions[9]
        return render_template("main.html", content = "Suggestions:  " + string, text = input_text)
    else:
        return render_template("main.html")

if __name__=='__main__':
    app.run()
    

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [03/Mar/2022 18:33:32] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [03/Mar/2022 18:33:37] "GET /main HTTP/1.1" 200 -
127.0.0.1 - - [03/Mar/2022 18:34:00] "POST /main HTTP/1.1" 200 -
127.0.0.1 - - [03/Mar/2022 18:34:50] "POST /main HTTP/1.1" 200 -
127.0.0.1 - - [03/Mar/2022 18:34:53] "POST /main HTTP/1.1" 200 -
127.0.0.1 - - [03/Mar/2022 18:35:00] "POST /main HTTP/1.1" 200 -
127.0.0.1 - - [03/Mar/2022 18:35:07] "POST /main HTTP/1.1" 200 -
